In [1]:
import pandas as pd
import numpy as np

In [2]:
companies_selected = pd.read_excel("Companies.xlsx")

In [3]:
companies_selected.head()

,Company Name,Industry,Beta Value,Grade,TSR
0,NextEra Energy (NEE),Utilities,0.49,B,53.47
1,Duke Energy (DUK),Utilities,0.38,C,57.51
2,Atmos Energy (ATO),Utilities,0.50,B,54.13
3,Avangrid (AGR),Utilities,0.34,A,-8.57
4,Otter Tail (OTTR),Utilities,0.69,A,87.72


In [3]:
# since TSR is in % we will grade wight in % as well
grade_weight = {"A": 90, "B": 80, "C": 70, "D": 60, "E": 50, "F": 40}
companies_selected["GradeWeight"] = companies_selected["Grade"].map(grade_weight)

In [4]:
def calculate_grade_score(data, x):
    return (1 - x) * data["TSR"] + x * data["GradeWeight"]

In [5]:
def monte_carlo_simulation(
    data, max_industry_allocation=0.16, max_stock_allocation=0.045, target_beta=1.0
):
    best_portfolio = None
    best_portfolio_beta = float("inf")
    best_run = None

    for iteration in range(65000, 80000):  # We want to keep the grade wight at a minumum of 0.65 of the total caluclation
        weight_factor = iteration / 100000  # Fraction for Grade Score calculation

        # Normalize TSR:
        data["Industry TSR total"] = data.groupby("Industry")["TSR"].transform("sum")
        data["Weighted TSR"] = data["TSR"] / data["Industry TSR total"]

        # Calculate Grade Score:
        data["Grade Score"] = data.apply(
            calculate_grade_score, axis=1, args=(weight_factor,)
        )

        # Calculate Weight:
        data["Weight"] = data["Grade Score"] / data["Grade Score"].sum()

        # Apply industry-level restrictions:
        data["Industry Allocation"] = data.groupby("Industry")["Weight"].transform(
            "sum"
        )
        data["Allocation_by_restriction"] = np.where(
            data["Industry Allocation"] > max_industry_allocation,
            max_industry_allocation,
            data["Industry Allocation"],
        )

        # Normalize Weights based on restrictions:
        data["Normalised Weight"] = np.where(
            data["Allocation_by_restriction"] == max_industry_allocation,
            (data["Weight"] / data["Industry Allocation"]) * max_industry_allocation,
            data["Weight"],
        )

        # Calculate Portfolio Beta:
        portfolio_beta = (data["Beta Value"] * data["Normalised Weight"]).sum()

        # Check if this portfolio is better:
        if portfolio_beta < best_portfolio_beta:
            test = weight_factor
            best_portfolio_beta = portfolio_beta
            best_run = iteration
            best_portfolio = data.copy()
            print(best_run, best_portfolio_beta)

    return test, best_run, best_portfolio_beta, best_portfolio

In [6]:
test, run_no, final_beta, ideal_portfolio = monte_carlo_simulation(companies_selected)
print(f"best run: {run_no} final_beta: {final_beta} ")
print(ideal_portfolio)

65000 0.8796795000604941
best run: 65000 final_beta: 0.8796795000604941 
                                Company Name          Industry  Beta Value  \
0                       NextEra Energy (NEE)         Utilities        0.49   
1                          Duke Energy (DUK)         Utilities        0.38   
2                         Atmos Energy (ATO)         Utilities        0.50   
3                             Avangrid (AGR)         Utilities        0.34   
4                          Otter Tail (OTTR)         Utilities        0.69   
..                                       ...               ...         ...   
74  COSTCO WHOLESALE CORPORATION (XNAS:COST)  Consumer Staples        0.79   
75      DOLLAR GENERAL CORPORATION (XNYS:DG)  Consumer Staples        0.45   
76       COLGATE-PALMOLIVE COMPANY (XNYS:CL)  Consumer Staples        0.41   
77            THE HERSHEY COMPANY (XNYS:HSY)  Consumer Staples        0.37   
78                    UNILEVER PLC (XBUE:UL)  Consumer Staples       

In [7]:
test

0.65

In [8]:
run_no

65000

In [9]:
final_beta

0.8796795000604941

In [10]:
ideal_portfolio

,Company Name,Industry,Beta Value,Grade,TSR,GradeWeight,Industry TSR total,Weighted TSR,Grade Score,Weight,Industry Allocation,Allocation_by_restriction,Normalised Weight
0,NextEra Energy (NEE),Utilities,0.49,B,53.47,80,358.64,0.149091,70.7145,0.009452,0.087150,0.087150,0.009452
1,Duke Energy (DUK),Utilities,0.38,C,57.51,70,358.64,0.160356,65.6285,0.008772,0.087150,0.087150,0.008772
2,Atmos Energy (ATO),Utilities,0.50,B,54.13,80,358.64,0.150931,70.9455,0.009483,0.087150,0.087150,0.009483
3,Avangrid (AGR),Utilities,0.34,A,-8.57,90,358.64,-0.023896,55.5005,0.007418,0.087150,0.087150,0.007418
4,Otter Tail (OTTR),Utilities,0.69,A,87.72,90,358.64,0.244591,89.2020,0.011923,0.087150,0.087150,0.011923
...,...,...,...,...,...,...,...,...,...,...,...,...,...
74,COSTCO WHOLESALE CORPORATION (XNAS:COST),Consumer Staples,0.79,D,17.43,60,177.69,0.098092,45.1005,0.006028,0.077816,0.077816,0.006028
75,DOLLAR GENERAL CORPORATION (XNYS:DG),Consumer Staples,0.45,B,14.09,80,177.69,0.079295,56.9315,0.007609,0.077816,0.077816,0.007609
76,COLGATE-PALMOLIVE COMPANY (XNYS:CL),Consumer Staples,0.41,A,15.40,90,177.69,0.086668,63.8900,0.008540,0.077816,0.077816,0.008540
77,THE HERSHEY COMPANY (XNYS:HSY),Consumer Staples,0.37,B,12.95,80,177.69,0.072880,56.5325,0.007556,0.077816,0.077816,0.007556
